Note 1. VmA2D & VmD2A for now using Vm; in MEND, they are modified by water
        temperature scalar
        sPAR % VmA2D = sPARbase%Vm * tp_scalar*wp_scalar_low (L2108)
        sPAR % VmD2A = sPARbase%Vm * tp_scalar*wp_scalar (L2109)

In [1]:
using UnPack


In [2]:
## 1 develop struct for soil 
Base.@kwdef mutable struct SoilPool
    carbon::Float64 
    nitrogen::Float64 
    phosphorus::Float64 
end
Base.@kwdef mutable struct SoilPar
    "parameters"
    fRa::Float64 = 0.2
    fINP::Float64 = 0.3
    vd_pomo::Float64   = 50.0
    vd_pomh::Float64   = 50.0
    vd_mom::Float64    = vd_pomh * 0.01
    ks_pomo::Float64   = 60.0
    fKM::Float64       = 10.0
    ks_pomh::Float64   = ks_pomo/fKM
    ks_mom::Float64    = ks_pomo * fKM
    Qmax::Float64      = 1.5
    Kba::Float64       = 6.0 
    Kdes::Float64      = 0.006 
    Kads::Float64      = Kdes * Kba    
    Kp2u::Float64      = 0.000005
    Ku2p::Float64      = 0.001
    rENZM::Float64     = 0.00012289
    rENZPo::Float64    = rENZM
    rENZPh::Float64    = rENZM
    pENZP::Float64     = 0.00147568
    fpEM::Float64      = 4.50361918
    pENZM::Float64     = pENZP * fpEM    
    frPOM2DOM::Float64 = 0.75
    frMB2DOM::Float64  = 0.5
    frMBA_to_POMo::Float64 = 0.1
    frMBA_to_POMh::Float64 = 0.9
    Vg::Float64        = 0.00425194
    alpha::Float64     = 0.05012233
    Vm::Float64        = Vg * alpha/(1.0 - alpha)
    KsDOM::Float64     = 0.00010034
    Yg::Float64        = 0.20109320 
    Ygsl::Float64      = 0.00520677
    CUE_slope::Float64 = -1.0*Ygsl
    Q10::Float64       = 1.8
    gamma::Float64     = 0.01030726
    rMORT::Float64     = min(0.99,Vm * gamma)
    beta::Float64      = 0.001
    VmD::Float64       = Vm * beta
    VmA2D::Float64     = Vm #* tp_scalar * wp_scalar_low
    VmD2A::Float64     = Vm #* tp_scalar * wp_scalar
    SWP_A2D::Float64   = 0.46
    tau::Float64       = 0.39
    SWP_D2A::Float64   = tau * SWP_A2D
    wdorm::Float64     = 3.38
    VNup_MB::Float64   = 0.1
    VNup_VG::Float64   = 0.00003296
    rNleach::Float64   = 0.02
    bNup_VG::Float64   = 0.5
    KsNH4_MB::Float64  = 0.00018
    KsNO3_MB::Float64  = 0.00041
    # YgN = sPAR%YgN        = phi 
    Qmax_NH4::Float64  = 0.0057442
    Kba_NH4::Float64   = 100.0
    KsNH4_VG::Float64  = 0.0012
    KsNO3_VG::Float64  = 0.0018
    fpENZN::Float64    = 1.0
    VNif::Float64      = 0.0635041
    VNit::Float64      = 185.28188371
    VDenit::Vector{Float64} = [0.86952628,0.86952628,0.86952628,0.86952628]
    # VDenit(1) = 0.86952628
    # VDenit(2) = 0.86952628
    # VDenit(3) = 0.86952628
    # VDenit(4) = 0.86952628
    KsNif::Float64 = 0.1
    KsNit::Float64 = 0.0012
    KsDenit::Vector{Float64} = [0.0018,0.0018,0.0018,0.0018]
    # KsDenit(1) = 0.0018
    # KsDenit(2) = 0.0018
    # KsDenit(3) = 0.0018
    # KsDenit(4) = 0.0018 
    
end

Base.@kwdef mutable struct POMo
    carbon::Float64
    nitrogen::Float64
    phosphorus::Float64
    enzyme::Float64
end

Base.@kwdef mutable struct POMh
    carbon::Float64
    nitrogen::Float64
    phosphorus::Float64
    enzyme::Float64
end

Base.@kwdef mutable struct MOM
    carbon::Float64
    nitrogen::Float64
    phosphorus::Float64
    enzyme::Float64 
end

Base.@kwdef mutable struct DOM
    carbon::Float64
    nitrogen::Float64
    phosphorus::Float64
    # enzyme::Float64 
end

Base.@kwdef mutable struct QOM
    carbon::Float64
    nitrogen::Float64
    phosphorus::Float64
end

Base.@kwdef mutable struct MBA
    carbon::Float64
    nitrogen::Float64
    phosphorus::Float64
end

Base.@kwdef mutable struct MBD
    carbon::Float64
    nitrogen::Float64
    phosphorus::Float64
end

MBD

In [3]:
## 2 develop functions: Michaelis Menten

function MM(par::SoilPar,pool::POMo) 
    @unpack vd_pomo, ks_pomo  = par
    @unpack carbon, enzyme = pool
    vm = vd_pomo   
    km = ks_pomo
    substrate = carbon
        
    MM = vm * substrate * enzyme/(km + substrate)
    MM = min(MM, substrate)
    return MM
end

function MM(par::SoilPar,pool::POMh) 
    @unpack vd_pomh, ks_pomh  = par
    @unpack carbon, enzyme = pool
    vm = vd_pomh   
    km = ks_pomh
    substrate = carbon
        
    MM = vm * substrate * enzyme/(km + substrate)
    MM = min(MM, substrate)
    return MM
end

function MM(par::SoilPar,pool::MOM) 
    @unpack vd_mom, ks_mom  = par
    @unpack carbon, enzyme = pool
    vm = vd_mom   
    km = ks_mom
    substrate = carbon
        
    MM = vm * substrate * enzyme/(km + substrate)
    MM = min(MM, substrate)
    return MM
end

function MM(par::SoilPar,pool::DOM) 
    @unpack Vg, Vm, Yg, KsDOM  = par
    @unpack carbon, enzyme = pool
    vm = (Vg + Vm)/Yg    
    km = KsDOM
    substrate = carbon
        
    MM = vm * substrate * enzyme/(km + substrate)
    MM = min(MM, substrate)
    return MM
end

function MM(par::SoilPar,dom::DOM,mba::MBA) 
    @unpack Vg, Yg, KsDOM, Vm  = par
    enzyme    = mba.carbon 
    substrate = dom.carbon
    vm = Vg * (1.0/Yg - 1.0)   
    km = KsDOM
        
    MM_growth = vm * substrate * enzyme/(km + substrate)
    MM_growth = min(MM_growth, substrate)

    vm = Vm * (1.0/Yg - 1.0)
    MM_maintn = vm * substrate * enzyme/(km + substrate)
    MM_maintn = min(MM_maintn, substrate)
    return MM_growth, MM_maintn
end


MM (generic function with 5 methods)

In [4]:
## 3 develop functions: Fluxes

function Flux(par::SoilPar,pool::POMo)
    @unpack frPOM2DOM = par
    pom_dec = MM(par,pool)
    pom_dom = frPOM2DOM * pom_dec
    pom_mom = (1.0 - frPOM2DOM) * pom_dec  
    return pom_dom,pom_mom  
end

function Flux(par::SoilPar,pool::POMh)
    @unpack frPOM2DOM = par
    pom_dec = MM(par,pool)
    pom_dom = frPOM2DOM * pom_dec
    pom_mom = (1.0 - frPOM2DOM) * pom_dec 
    return pom_dom,pom_mom   
end

function Flux(par::SoilPar,mom::MOM)
    mom_dec = MM(par,mom)
    mom_dom = mom_dec
    return mom_dom 
end

function Flux(par::SoilPar,dom::DOM, qom::QOM)
    dom_dec = MM(par,dom)
    dom_mba = dom_dec 
    dom.carbon = dom.carbon - dom_mba # the preference given to microbial uptake then ad_de
    ads,des = AdsDesorption(par, dom::DOM, qom::QOM)
    dom_qom = ads  
    qom_dom = des
    return dom_qom, qom_dom 
end

function Flux(par::SoilPar,mba::MBA,dom::DOM)
    @unpack rMORT, frMB2DOM, frMBA_to_POMh, frMBA_to_POMo, KsDOM, VmA2D = par
    @unpack carbon = mba 
    mb = carbon
    mba_mortality= rMORT * mb
    mba_dom   = frMB2DOM * mba_mortality
    mba_pomh   = (1.0 - frMB2DOM) * mba_mortality * frMBA_to_POMh
    mba_pomo   = (1.0 - frMB2DOM) * mba_mortality * frMBA_to_POMo
    
    phi = dom.carbon/(dom.carbon + KsDOM)
    mba_mbd = (1.0 - phi) * VmA2D * mb
    mba_CO2_growth, mba_CO2_maintn = MM(par,mba,dom) # Respiration of MBA
end

function Flux(par::SoilPar,mbd::MBD,dom::DOM) # Respiration of MBD and resurcitaion
    @unpack KsDOM,VmD2A,VmD = par
    phi = dom.carbon/(dom.carbon + KsDOM)
    mbd_mba = phi * VmD2A * mbd.carbon
    mbd_CO2_maintn = VmD * mbd.carbon 
end



Flux (generic function with 6 methods)

In [5]:
## 3 developing functions: assorted
# adsorption and desorption; microbial dormancy and resuscitation


function AdsDesorption(par, pool1::DOM, pool2::QOM)
    @unpack Kads,Qmax,Kdes = par
    @unpack carbon = pool1
    adsorbate = carbon
    @unpack carbon = pool2 
    adsorbent = carbon
    ads = Kads * adsorbate * (1.0 - adsorbent/Qmax)
    des = Kdes * adsorbent / Qmax
    if des > (adsorbent + ads)
        des = adsorbent + ads 
    elseif ads > adsorbate + des 
        ads = adsorbate + des
    end 
    return ads,des 
end

# function DorResus()
# end

function EnzymeProduction(par::SoilPar,pool::MBA,pomo::POMo,pomh::POMh)
    @unpack pENZP, Vm, pENZM = par
    @unpack carbon = pool 
    mb = carbon 
    frPOMh = pomh.carbon/(pomh.carbon + pomo.carbon)
    mba_enzph = frPOMh * pENZP * Vm * mb
    mba_enzpo = (1-frPOMh) * pENZP * Vm * mb
    mba_enzm =  pENZM * Vm * mb
    return mba_enzph, mba_enzpo, mab_enzm 
end 

function EnzymeTurnover(par::SoilPar,pomo::POMo)
    epo_dom = par.rENZPo * pomo.enzyme
end

function EnzymeTurnover(par::SoilPar,pomh::POMh)
    eph_dom = par.rENZPh * pomh.enzyme
end

function EnzymeTurnover(par::SoilPar,mom::MOM)
    em_dom  = par.rENZM * mom.enzyme
end

EnzymeProduction (generic function with 1 method)

In [ ]:
# Base.@kwdef mutable struct POMo0
#     carbon::Float64=1
#     nitrogen::Float64
#     phosphorus::Float64
#     enzyme::Float64
# end

In [6]:
## 4 Initial condition & carbon input
pomo = POMo(4000.,400.,40.,2.)
pomh = POMh(2000.,100.,5.,1.)
mom = MOM(5000.,1000.,200.,10.)
dom = DOM(200., 10., 1.)
qom = QOM(500.,100.,20.)
mba = MBA(1000.,150.,15.,)
mbd = MBD(1000.,150.,15.,)

litter_pomo = 1000.0/365.0/24.0 * 0.5
litter_pomh = 1000.0/365.0/24.0 * 0.25
litter_dom = 1000.0/365.0/24.0 * 0.25


MBD(1000.0, 150.0, 15.0)

In [10]:
# Temporal dynamics
for iday = 1:15
    for ihour = 1:24
        pomo = pomo - pomo_dec + mba_pomo + litter_pomo
        pomh = pomh - pomh_dec + mba_pomh + litter_pomh 
        mom = mom - mom_dec + pomo_mom + pomh_mom 
        dom = (dom - dom_dec + qom_dom + litter_dom + pomh_dom 
                + pomo_dom + mba_dom + epo_dom + eph_dom + em_dom)
        qom = qom - qom_dom + dom_qom 
        mba = (mba - mba_mortality - mba_mbd - mba_CO2_growth - mba_CO2_maintn
                + dom_mba + mbd_mba)
        mbd = mbd - mbd_mba - mbd_CO2_maintn + mba_mbd
    end
end

1.5

In [9]:
(1+2
+3+4
+4)

14